In [1]:
### install libraries
!pip install pretty_midi
!pip install scikit-learn

     |████████████████████████████████| 5.6 MB 4.1 MB/s 
     |████████████████████████████████| 51 kB 6.3 MB/s 
  Created wheel for pretty-midi: filename=pretty_midi-0.2.9-py3-none-any.whl size=5591953 sha256=f88011b4f5f983744cb68119a2a4e11258d799a331888a404a7b8a93f1b8b079
  Stored in directory: /root/.cache/pip/wheels/ad/74/7c/a06473ca8dcb63efb98c1e67667ce39d52100f837835ea18fa
Successfully built pretty-midi


In [2]:
### mount drive
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [19]:
### import libraries
import numpy as np
import pandas as pd
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
import pretty_midi
import os
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
import warnings
warnings.filterwarnings('ignore')

In [4]:
##### Function to get all the midi file paths in a directory in a dataframe.
def get_midi_paths(midi_folder):

    file_paths = []
    for dir_name, subdir_list, file_list in os.walk(midi_folder):
        file_path_list = ["/".join([dir_name, file]) for file in file_list]
        for file_path in file_path_list:
            file_paths.append(file_path)
    df = pd.DataFrame({"Path": file_paths})
    return df


In [7]:
## get the paths for the generate midi files
midi_path_gen = "//content/drive/MyDrive/music_data/midi/generated_midi"
gen_midi_df = get_midi_paths(midi_path_gen)
print(gen_midi_df.size)
print(gen_midi_df.head())


1600
                                                Path
0  //content/drive/MyDrive/music_data/midi/genera...
1  //content/drive/MyDrive/music_data/midi/genera...
2  //content/drive/MyDrive/music_data/midi/genera...
3  //content/drive/MyDrive/music_data/midi/genera...
4  //content/drive/MyDrive/music_data/midi/genera...


In [12]:
## get the paths for the original or real music midi files
midi_path_og = "/content/drive/MyDrive/music_data/midi/adl_piano"
og_midi_df = get_midi_paths(midi_path_og)
print(og_midi_df.size)
print(og_midi_df.head())


1668
                                                Path
0  /content/drive/MyDrive/music_data/midi/adl_pia...
1  /content/drive/MyDrive/music_data/midi/adl_pia...
2  /content/drive/MyDrive/music_data/midi/adl_pia...
3  /content/drive/MyDrive/music_data/midi/adl_pia...
4  /content/drive/MyDrive/music_data/midi/adl_pia...


In [8]:
### FEATURE EXTRACTION

## function to normalize features
def normalize_features(features):

    tempo = (features[0] - 150) / 300
    num_sig_changes = (features[1] - 2) / 10
    resolution = (features[2] - 260) / 400
    time_sig_1 = (features[3] - 3) / 8
    time_sig_2 = (features[4] - 3) / 8
    return [tempo, resolution, time_sig_1, time_sig_2]

### extracting featres
def get_features(path):
    try:
        # Test for Corrupted Midi Files
        with warnings.catch_warnings():
            warnings.simplefilter("error")
            file = pretty_midi.PrettyMIDI(path)
            
            tempo = file.estimate_tempo()
            num_sig_changes = len(file.time_signature_changes)
            resolution = file.resolution
            ts_changes = file.time_signature_changes
            ts_1 = 4
            ts_2 = 4
            if len(ts_changes) > 0:
                ts_1 = ts_changes[0].numerator
                ts_2 = ts_changes[0].denominator
            return normalize_features([tempo, num_sig_changes, resolution, ts_1, ts_2])
    except:
        return None

### function to traverse the whole dataframe to get all the features for all the midi files
def extract_midi_features(path_df):
    all_features = []
    for index, row in path_df.iterrows():
        features = get_features(row.Path)
        if features is not None:
            all_features.append(features)
    return np.array(all_features)



In [9]:
## extracting features for generated files
gen_features = extract_midi_features(gen_midi_df)
print(gen_features)

[[-0.34722222 -0.1         0.125       0.125     ]
 [ 0.13520267 -0.1         0.125       0.125     ]
 [ 0.17822736 -0.1         0.125       0.125     ]
 ...
 [ 0.12775616 -0.1         0.125       0.125     ]
 [-0.05502677 -0.1         0.125       0.125     ]
 [-0.06489493 -0.1         0.125       0.125     ]]


In [10]:
#### ADDING label = 0 to the generated music data points
#print(gen_midi_df.shape)
label_gen = np.zeros((gen_features.shape[0], 1))
print(label_gen.shape)
print(gen_features.shape)
gen_feat = np.append(gen_features, label_gen, axis =1)
print(gen_feat.shape)

(1599, 1)
(1599, 4)
(1599, 5)


In [13]:
## extracting features for real music files
og_features = extract_midi_features(og_midi_df)
print(og_features)

[[-0.06515116 -0.35        0.625       0.125     ]
 [ 0.14770241 -0.05        0.125       0.125     ]
 [ 0.1046981  -0.35        0.125       0.125     ]
 ...
 [ 0.00224215 -0.41        0.125       0.125     ]
 [ 0.18500032 -0.17        0.125       0.125     ]
 [ 0.03631285  0.31        0.125       0.125     ]]


In [14]:
#### ADDING label = 0 to the greal music data points
label_og = np.ones((og_features.shape[0], 1))
print(label_og.shape)
print(og_features.shape)
og_feat = np.append(og_features, label_og, axis =1)
print(og_feat.shape)

(1631, 1)
(1631, 4)
(1631, 5)


In [15]:
### appending both to get a single df
df_c = np.append(og_feat, gen_feat, axis = 0)
print(df_c.shape)

(3230, 5)


In [16]:
#shuffling to mix up the samples
df_c = np.random.permutation(df_c)
df_c

array([[ 0.05841732, -0.1       ,  0.125     ,  0.125     ,  0.        ],
       [ 0.02747253, -0.1       ,  0.125     ,  0.125     ,  0.        ],
       [ 0.12278839, -0.1       ,  0.125     ,  0.125     ,  0.        ],
       ...,
       [ 0.24105263, -0.1       ,  0.125     ,  0.125     ,  0.        ],
       [ 0.07894737, -0.1       ,  0.125     ,  0.125     ,  0.        ],
       [-0.0210596 , -0.35      ,  0.125     ,  0.125     ,  1.        ]])

In [17]:

# SPLITTING the df into 3 parts
num = len(df_c)
num_training = int(num * 0.6)
num_validation = int(num * 0.8)
training_data = df_c[:num_training]
validation_data =  df_c[num_training:num_validation]
test_data =  df_c[num_validation:]

# Separating features and labels
num_cols = training_data.shape[1] - 1
training_features = training_data[:, :num_cols]
validation_features = validation_data[:, :num_cols]
test_features = test_data[:, :num_cols]

# formatting features
num_classes = 2
training_labels = training_data[:, num_cols].astype(int)
validation_labels = validation_data[:, num_cols].astype(int)
test_labels = test_data[:, num_cols].astype(int)


print(test_features[:10])
print(test_labels[:10])

[[-0.06666667 -0.01       -0.125       0.125     ]
 [-0.08107706 -0.1         0.125       0.125     ]
 [ 0.1        -0.01        0.          0.125     ]
 [ 0.25495979 -0.1         0.125       0.125     ]
 [-0.27943844  0.31        0.125       0.125     ]
 [-0.01020408 -0.1         0.125       0.125     ]
 [ 0.16666667 -0.01        0.125       0.125     ]
 [ 0.1547619  -0.1         0.125       0.125     ]
 [ 0.22781745  0.55       -0.125       0.125     ]
 [ 0.0312843  -0.1         0.125       0.125     ]]
[1 0 1 0 1 0 1 0 1 0]


TRAIN MODELS

In [20]:
#####Train multiple models and choose the one that gives the best results. We can also comment out the classifiers to see the individual performance.

def train_model(t_features, t_labels, v_features, v_labels):

    # Neural Network and SVM Configurations
    clf_1 = MLPClassifier(solver='adam', alpha=1e-4, hidden_layer_sizes=(5,), random_state=1)
    clf_2 = MLPClassifier(solver='adam', alpha=1e-4, hidden_layer_sizes=(5, 5), random_state=1)
    clf_3 = MLPClassifier(solver='adam', alpha=1e-5, hidden_layer_sizes=(10, 10), random_state=1)
    clf_4 = KNeighborsClassifier(n_neighbors=3)
    clf_5 = KNeighborsClassifier(n_neighbors=5)
    clf_6 = DecisionTreeClassifier(max_depth=1)

    clf_svm = SVC()
    
    
    # Keep Track of the Best Model
    best_clf = None
    best_accuracy = 0
    
    # Test the Accuracies of the Models and Get Best
    for clf in [clf_1, clf_2, clf_3, clf_4, clf_5, clf_6, clf_svm]:
  
        clf = clf.fit(t_features, t_labels)
        predictions = clf.predict(v_features)
        count = 0
        for i in range(len(v_labels)):
              if v_labels[i] == predictions[i]:
                  count += 1
        accuracy = count / len(v_labels)
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_clf = clf

    print("Best Training Accuracy:", best_accuracy)
    return best_clf

classifier = train_model(training_features, training_labels, validation_features, validation_labels)

Best Training Accuracy: 0.9984520123839009


In [21]:
######Use to visualise the tree if the classifier is the decision tree
#from sklearn import tree
#tree.plot_tree(classifier)

In [22]:
### find out the test accuracy
def test_accuracy(clf, t_features, t_labels):
    count = 0
    predictions = clf.predict(t_features)
    t_labels_hot = t_labels
    for i in range(len(t_features)):
        if (type(clf) == SVC):
            if t_labels[i] == predictions[i]:
                count += 1
        else:
            if np.array_equal(t_labels_hot[i], predictions[i]):
                count += 1
    return count / len(t_features)

print("TEST ACCURACY:", test_accuracy(classifier, test_features, test_labels))

TEST ACCURACY: 0.9984520123839009
